### Read MNIST data

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import MNISTtools
xtrain, ltrain = MNISTtools.load(dataset = "training", path = "/datasets/MNIST")

### Shape, Size and Feature Dimension

In [71]:
example_num = xtrain.shape[1]
print( "The shape of xtrain is: %s" %(xtrain.shape,) )
print( "The shape of ltrain is: %s" %(ltrain.shape,) )
print( "The size of training dataset is: %s" %ltrain.size )
print( "The feature dimension is: %s" %xtrain.shape[0] )
print("Range of xtrain pixels is: [%s,%s]" %(np.min(xtrain),np.max(xtrain)))

The shape of xtrain is: (784, 60000)
The shape of ltrain is: (60000,)
The size of training dataset is: 60000
The feature dimension is: 784
Range of xtrain pixels is: [0,255]


### Normalize

In [72]:
def normalize_MNIST_images(x):
    """
    :param x: a collection of images
    :type x: np.array int8
    :return: modified version of images [-1,1]
    :rtype: np.array float32
    """
    return ((x-127.5)/127.5).astype(np.float32)

### Shuffle the dataset

In [73]:
shuffled_ind = np.random.permutation(example_num)
print("Range of index is: [%s,%s]" %(np.min(shuffled_ind),np.max(shuffled_ind)))
print(shuffled_ind)
xtrain = xtrain[:,shuffled_ind]
ltrain = ltrain[shuffled_ind]
print(xtrain.shape)
print(ltrain.shape)

Range of index is: [0,59999]
[28535  2425 31460 ... 47361 52534 24646]
(784, 60000)
(60000,)


### check output after shuffling and normalization

In [74]:
xtrain = normalize_MNIST_images(xtrain)
print("Range of xtrain after normalization is: [%s,%s]" %(np.min(xtrain),np.max(xtrain)))

Range of xtrain after normalization is: [-1.0,1.0]


### Create holdout / validation set 

In [75]:
valid_num = int(example_num / 10)
xvalid = xtrain[:,(example_num - valid_num):example_num]
lvalid = ltrain[(example_num - valid_num):example_num]
print(xvalid.shape,lvalid.shape)
xtrain = xtrain[:,:example_num - valid_num]
ltrain = ltrain[:example_num - valid_num]
print(xtrain.shape,ltrain.shape)

(784, 6000) (6000,)
(784, 54000) (54000,)


### label2onehot function

In [76]:
def label2onehot(lbl):
    d = np.zeros((lbl.max() + 1, lbl.size)) # d.shape: (9+1,60000) = (10,60000)
    d[lbl, np.arange(0, lbl.size)] = 1
    return d

dtrain = label2onehot(ltrain)
dvalid = label2onehot(lvalid)
print("One-hot code dtrain[:,42] is: %s and the label it predicts is: %s" %(dtrain[:,42],np.argwhere(dtrain[:,42]==1)[0][0]))
print(dtrain.shape,dvalid.shape)

One-hot code dtrain[:,42] is: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] and the label it predicts is: 2
(10, 54000) (10, 6000)


### onehot2label function

In [77]:
def onehot2label(d):
    lbl = d.argmax(0)# return the index of max element for every column
    return lbl

In [78]:
def softmax(a):
    return torch.exp(a - a.max(0)[0]) / (torch.exp(a - a.max(0)[0]).sum(dim = 0))

### softmax

In [79]:
def softmaxp(a, e):
    g = softmax(a)
    product = (g * e).sum(0)
    return g * e - product * g

In [80]:
import torch

xtrain = torch.from_numpy(xtrain)
dtrain = torch.from_numpy(dtrain)
xvalid = torch.from_numpy(xvalid)
dvalid = torch.from_numpy(dvalid)
ltrain = torch.from_numpy(ltrain)
lvalid = torch.from_numpy(lvalid)
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
print(device)
xtrain,dtrain = xtrain.to(device), dtrain.to(device)
xvalid,dvalid = xvalid.to(device), dvalid.to(device)
ltrain,lvalid = ltrain.to(device), lvalid.to(device)


cuda


### Initialization

In [81]:
def init_shallow(Ni, No, mu, sig):

    b = np.random.randn(No, 1) / np.sqrt((Ni+1.)/2.)
    #W = np.random.normal(mu, sig, (No,Ni))
    W = np.random.randn(No, Ni) / np.sqrt((Ni+1.)/2.)
    b, W = torch.from_numpy(b), torch.from_numpy(W)
    b, W = b.to(device), W.to(device)

    return W, b 

In [82]:
Ni = xtrain.shape[0] # 784
No = dtrain.shape[0] # 10
mu, sig = 0, 1e-3
netinit = init_shallow(Ni,No, mu, sig)

### eval_loss function: $E = - \sum_{i=1}^{10}d_{i}logy_{i}$
#### Compute averge cross-entropy loss (averaged over both training samples and vector dimension)

In [83]:
def eval_loss(y,d):
    return - (d * torch.log(y)).sum().double() / y.shape[1]

In [84]:
def risk(y,d):
    return - (d * np.log(y)).sum() 

### Function computes the percentage of misclassified samples between predictions and desired labels

In [85]:
def eval_perfs(y, lbl):
    return torch.sum(y.argmax(0)==lbl).double() / y.shape[1]

In [86]:
def forwardprop_shallow(x, net):
    W = net[0]
    b = net[1]
    a = W.double().mm(x.double()) + b 
    y = softmax(a) 
    return y

In [18]:
yinit = forwardprop_shallow(xtrain, netinit)
#print(torch.sum(torch.argmax(yinit,0)==ltrain).double()/54000)

In [19]:
"""
netinit = init_shallow(784,10, mu, sig)
W = netinit[0]
b = netinit[1]
a = W.double().mm(torch.cat((xtrain,xvalid),1).double()) + b 
y = softmax(a) 
print(eval_loss(y, torch.cat((dtrain,dvalid),1)), 'should be around .26')
"""

"\nnetinit = init_shallow(784,10, mu, sig)\nW = netinit[0]\nb = netinit[1]\na = W.double().mm(torch.cat((xtrain,xvalid),1).double()) + b \ny = softmax(a) \nprint(eval_loss(y, torch.cat((dtrain,dvalid),1)), 'should be around .26')\n"

In [87]:
def update_shallow(x, d, net, gamma=.05):
    W = net[0] # network param initialization
    b = net[1]
    Ni = W.shape[1] 
    No = W.shape[0]
    #gamma = gamma / x.shape[1] # normalized by the training dataset size
    
    # 1. Forward Process
    a = W.double().mm(x.double()) + b # W1:(10, 54k)
    y = softmax(a) # y:(10,54k)
    
    # 2. Compute Delta for Backprop
    delta = y - d # dE = -d/y, d2:(10,54k)
    
    # 3. Gradient Descent
    W -= gamma * delta.mm(x.t().double()) # gamma: learning rate
    b -= gamma * delta.sum(1).view(No,1) #(10,1)
    
    return W, b

In [21]:
train_ACC = []
validation_ACC = []
train_loss = []
validation_loss = []

In [22]:
def backprop_shallow(x, d, xv, dv, net, T, gamma_init):
    
    lbl = onehot2label(d)
    v_lbl = onehot2label(dv)
    
    for t in range(T):
        gamma = gamma_init / (1 + t / 32)
        net = update_shallow(x, d, net, gamma) # update net
        
        ypred = forwardprop_shallow(x, net)
        v_ypred = forwardprop_shallow(xv, net)
        loss = eval_loss(ypred,d) / 10 # training loss
        train_loss.append(loss)
        v_loss = eval_loss(v_ypred, dv) / 10 # validation loss
        validation_loss.append(v_loss)
        
        train_acc = eval_perfs(ypred,lbl)
        train_ACC.append(train_acc)
        validation_acc = eval_perfs(v_ypred,v_lbl)
        validation_ACC.append(validation_acc)
        if t % 50 == 0 or t == T-1:
            print("Epoch {0}: training loss = {1}, validation loss = {2}, training acc = {3}, validation acc = {4}, gamma = {5}".format(
                t, loss, v_loss, train_acc, validation_acc, gamma))
        
    return net

### T = 1024

In [23]:
nettrain = backprop_shallow(xtrain, dtrain, xvalid, dvalid, netinit, T = 1024, gamma_init = 1e-5)

Epoch 0: training loss = 1.8364758101179537, validation loss = 1.8725664600107643, training acc = 0.10366666666666667, validation acc = 0.09266666666666666, gamma = 1e-05
Epoch 50: training loss = 0.06718858744964232, validation loss = 0.06880759835501571, training acc = 0.8726666666666667, validation acc = 0.8666666666666666, gamma = 3.902439024390244e-06
Epoch 100: training loss = 0.055173646616544704, validation loss = 0.05646647108031322, training acc = 0.8857222222222222, validation acc = 0.8795, gamma = 2.4242424242424244e-06
Epoch 150: training loss = 0.049965423255049074, validation loss = 0.0511580263590832, training acc = 0.8913518518518518, validation acc = 0.8848333333333332, gamma = 1.7582417582417585e-06
Epoch 200: training loss = 0.04687602752285919, validation loss = 0.048043548958401544, training acc = 0.8951111111111111, validation acc = 0.8893333333333333, gamma = 1.3793103448275862e-06
Epoch 250: training loss = 0.044774970222587446, validation loss = 0.045946062043

In [24]:
t_acc = [t.to('cpu') for t in train_ACC]
v_acc = [v.to('cpu') for v in validation_ACC]
t_loss = [t.to('cpu') for t in train_loss]
v_loss = [v.to('cpu') for v in validation_loss]

In [25]:
import pickle
data = [t_acc,v_acc,t_loss,v_loss]
with open("data", 'wb') as sd:
        pickle.dump(data, sd)

In [26]:
xtest, ltest = MNISTtools.load(dataset = "testing", path = "/datasets/MNIST")
xtest = torch.from_numpy(xtest)
ltest = torch.from_numpy(ltest)
xtest, ltest = xtest.to(device), ltest.to(device)
ypred_ = forwardprop_shallow(xtest,nettrain)
test_acc = eval_perfs(ypred_,ltest)

In [27]:
test_acc

tensor(0.8531, device='cuda:0', dtype=torch.float64)

### Network weights and average images

In [28]:
xtrain,dtrain = xtrain.to(device), dtrain.to(device)
xvalid,dvalid = xvalid.to(device), dvalid.to(device)
ltrain,lvalid = ltrain.to(device), lvalid.to(device)

In [29]:
w,b = nettrain[0], nettrain[1]

In [30]:
cls_avg = []
for i in range(10):
    ind = (ltrain == i).nonzero()
    cls = xtrain[:,ind]
    avg = torch.mean(cls,1)
    cls_avg.append(avg)

In [31]:
c_avg = [i.reshape(28,28).to("cpu") for i in cls_avg]

In [32]:
w_avg = []
for i in range(10):
    a = w[i]
    w_avg.append(a.reshape(28,28))
nn_avg = [i.to("cpu") for i in w_avg]  

In [33]:
import pickle
data1 = [c_avg,nn_avg]
with open("avgimg", 'wb') as sd:
        pickle.dump(data1, sd)

### Hyperparameter for learning rate

In [24]:
def backprop_shallow1(x, d, xv, dv, net, T, gamma_init):
    
    lbl = onehot2label(d)
    v_lbl = onehot2label(dv)
    
    for t in range(T):
        gamma = gamma_init / (1 + t / 32)
        net = update_shallow(x, d, net, gamma) # update net
        
    return net

In [25]:
gamma_set = [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7]
training_acc = []
valid_acc = []
for g in gamma_set:
    Ni = xtrain.shape[0] # 784
    No = dtrain.shape[0] # 10
    mu, sig = 0, 1e-3
    netinit = init_shallow(Ni,No, mu, sig)
    nettrain = backprop_shallow1(xtrain, dtrain, xvalid, dvalid, netinit, T = 1024, gamma_init = g)
    y = forwardprop_shallow(xtrain, nettrain)
    v_y = forwardprop_shallow(xvalid, nettrain)
    train_acc = eval_perfs(y,ltrain)
    validation_acc = eval_perfs(v_y,lvalid)
    print(train_acc,validation_acc)
    training_acc.append(train_acc)
    valid_acc.append(validation_acc)
    

tensor(0.9080, device='cuda:0', dtype=torch.float64) tensor(0.8995, device='cuda:0', dtype=torch.float64)
tensor(0.9073, device='cuda:0', dtype=torch.float64) tensor(0.8998, device='cuda:0', dtype=torch.float64)
tensor(0.9079, device='cuda:0', dtype=torch.float64) tensor(0.8990, device='cuda:0', dtype=torch.float64)
tensor(0.9075, device='cuda:0', dtype=torch.float64) tensor(0.9000, device='cuda:0', dtype=torch.float64)
tensor(0.9083, device='cuda:0', dtype=torch.float64) tensor(0.9017, device='cuda:0', dtype=torch.float64)
tensor(0.8745, device='cuda:0', dtype=torch.float64) tensor(0.8762, device='cuda:0', dtype=torch.float64)
tensor(0.7148, device='cuda:0', dtype=torch.float64) tensor(0.7208, device='cuda:0', dtype=torch.float64)


In [48]:
print(a1)

[tensor(0.9038, dtype=torch.float64), tensor(0.9071, dtype=torch.float64), tensor(0.9081, dtype=torch.float64), tensor(0.9064, dtype=torch.float64), tensor(0.9071, dtype=torch.float64), tensor(0.8716, dtype=torch.float64), tensor(0.6947, dtype=torch.float64)]


In [47]:
a1 = [i.to('cpu') for i in training_acc]
a2 = [i.to('cpu') for i in valid_acc]
data2 = [a1, a2]
with open("eta","wb") as f:
    pickle.dump(data2,f)

<h2><center>20</center></h2>

### 20.1 Implement the Backpropagation based on SGD/Minibatch Gradient Descent 

In [88]:
def backprop_minibatch_shallow(x, d, v, dv, net, T, B=64, gamma_init=.05):

    N = x.shape[1] # sample number
    NB = int((N + B - 1)/B) # = N/B + 1 - 1/B 
    lbl = onehot2label(d)
    v_lbl = onehot2label(dv)
    
    for t in range(0, T):
        shuffled_indices = torch.randperm(N)
        gamma = gamma_init / (1 + t / 8)
        
        # mini-batch learning with NB updates
        for l in range(NB):
            minibatch_indices = shuffled_indices[B*l : min(B*(l+1), N)] # the indices of training samples == the indices of columns
            net = update_shallow(x[:,minibatch_indices],d[:,minibatch_indices],net,gamma) # update net
        
        # report the loss and training error rates every epoch
        y = forwardprop_shallow(x, net)
        y_ = forwardprop_shallow(v, net)
        loss = eval_loss(y,d) / 10
        t_risk.append(loss)
        v_loss = eval_loss(y_,dv) / 10
        v_risk.append(v_loss)
        train_acc = eval_perfs(y, lbl)
        t_acc.append(train_acc)
        valid_acc = eval_perfs(y_, v_lbl)
        v_acc.append(valid_acc)
        print("Epoch {0}: train_loss= {1}, valid_loss= {2}, train_acc= {3}, valid_acc= {4}".format(t, loss, v_loss, train_acc, valid_acc))
    
    return net


In [89]:
g = 1e-3
mu, sig = 0, 1e-3
t_acc, v_acc = [], []
t_risk, v_risk = [], []
Ni = xtrain.shape[0] # 784
No = dtrain.shape[0] # 10
netinit = init_shallow(Ni, No, mu, sig)
netminibatch = backprop_minibatch_shallow(xtrain, dtrain, xvalid, dvalid, netinit, T = 8, B=64, gamma_init = g)    
    

Epoch 0: train_loss= 0.03319656546053445, valid_loss= 0.03089148802476522, train_acc= 0.9055925925925926, valid_acc= 0.9099999999999999
Epoch 1: train_loss= 0.031966146467132917, valid_loss= 0.030230005567258494, train_acc= 0.9056481481481481, valid_acc= 0.9053333333333333
Epoch 2: train_loss= 0.030425697646291463, valid_loss= 0.029620633154275516, train_acc= 0.9126851851851852, valid_acc= 0.9103333333333333
Epoch 3: train_loss= 0.03059660700616519, valid_loss= 0.029648231300550804, train_acc= 0.9107962962962963, valid_acc= 0.9119999999999999
Epoch 4: train_loss= 0.028257758341493147, valid_loss= 0.027407534517183862, train_acc= 0.9192222222222222, valid_acc= 0.9213333333333333
Epoch 5: train_loss= 0.028152146630412156, valid_loss= 0.02747860653541713, train_acc= 0.9197407407407407, valid_acc= 0.9208333333333333
Epoch 6: train_loss= 0.028543575987236426, valid_loss= 0.028148234668459178, train_acc= 0.9197777777777778, valid_acc= 0.9196666666666666
Epoch 7: train_loss= 0.027275542385565

In [60]:
import pickle
data4 = [t_acc,v_acc,t_risk,v_risk]
data4_ = []
for i in range(4):
    d = data4[i]
    d_ = [x.to("cpu") for x in d]
    data4_.append(d_)

with open("batchdata","wb") as f:
    pickle.dump(data4_,f)

In [93]:
xtest, ltest = MNISTtools.load(dataset = "testing", path = "/datasets/MNIST")
xtest = torch.from_numpy(xtest)
ltest = torch.from_numpy(ltest)
xtest, ltest = xtest.to(device), ltest.to(device)
ybatch = forwardprop_shallow(xtest,netminibatch)
test_acc = eval_perfs(ybatch,ltest)

In [94]:
test_acc

tensor(0.8543, device='cuda:0', dtype=torch.float64)